In [2]:
%run -i run_train.py \
    --gan_type StyleGanXL \
    --deformator ortho \
    --out ../BrainLatentDiscovery/res/

0
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


RuntimeError: CUDA out of memory. Tried to allocate 432.00 MiB (GPU 0; 12.00 GiB total capacity; 9.11 GiB already allocated; 0 bytes free; 9.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
from torch_tools.visualization import to_image
from visualization import interpolate
from loading import load_from_dir
%matplotlib inline


deformator, G, shift_predictor = load_from_dir(
    './models/pretrained/deformators/SN_MNIST/',
    G_weights='./models/pretrained/generators/SN_MNIST/')

# deformator, G, shift_predictor = load_from_dir(
#     './models/pretrained/deformators/SN_Anime/',
#     G_weights='./models/pretrained/generators/SN_Anime/')

# deformator, G, shift_predictor = load_from_dir(
#     './models/pretrained/deformators/BigGAN/',
#     G_weights='./models/pretrained/generators/BigGAN/G_ema.pth')

# deformator, G, shift_predictor = load_from_dir(
#     './models/pretrained/deformators/ProgGAN/',
#     G_weights='./models/pretrained/generators/ProgGAN/100_celeb_hq_network-snapshot-010403.pth')

# deformator, G, shift_predictor = load_from_dir(
#     './models/pretrained/deformators/StyleGAN2/',
#     G_weights='./models/pretrained/generators/StyleGAN2/stylegan2-ffhq-config-f.pt')

discovered_annotation = ''
for d in deformator.annotation.items():
    discovered_annotation += '{}: {}\n'.format(d[0], d[1])
print('human-annotated directions:\n' + discovered_annotation)

In [ ]:
from utils import is_conditional

rows = 8
plt.figure(figsize=(5, rows), dpi=250)

# set desired class for conditional GAN
if is_conditional(G):
    G.set_classes(12)

annotated = list(deformator.annotation.values())
inspection_dim = annotated[0]
zs = torch.randn([rows, G.dim_z] if type(G.dim_z) == int else [rows] + G.dim_z, device='cuda')


for z, i in zip(zs, range(rows)):
    interpolation_deformed = interpolate(
        G, z.unsqueeze(0),
        shifts_r=16,
        shifts_count=3,
        dim=inspection_dim,
        deformator=deformator,
        with_central_border=True)

    plt.subplot(rows, 1, i + 1)
    plt.axis('off')
    grid = make_grid(interpolation_deformed, nrow=11, padding=1, pad_value=0.0)
    grid = torch.clamp(grid, -1, 1)

    plt.imshow(to_image(grid))